# Club elo scraper
Store club elo per every 6 months as list of pandas dataframes
and store it as a csv file

In [8]:
from datetime import datetime
import io
import requests
import pandas as pd
from pathlib import Path
import time
import os
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup

In [2]:
# Generate dates with 6-month intervals starting from 2010-01-01 to today
date_range = pd.date_range(start='2023-01-01', end=pd.Timestamp.now(), freq='6ME')

# Convert the dates to the desired string format 'YYYY-MM-DD'
date_list = date_range.strftime('%Y-%m-%d').tolist()

In [9]:
date = date_list[-1]
# f"https://api.clubelo.com/{date}"
URL = f"https://api.clubelo.com/{date}"

In [10]:
res = Request(URL,headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.5',
    'Referer': 'http://google.com',  # Optional, mimics the referer if needed
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Connection': 'keep-alive'  # Helps maintain a persistent connection
})
html_page = urlopen(res).read()
soup = BeautifulSoup(html_page, 'html.parser')


URLError: <urlopen error [Errno 60] Operation timed out>

In [10]:
date = date_list[0]
response = requests.get(f"https://api.clubelo.com/{date}")
df = pd.read_csv(io.StringIO(response.content.decode('utf-8')))
df['Rank'] = df.index + 1
df['Updated'] = date


KeyboardInterrupt: 

In [8]:

dfs = []
merged_df = pd.DataFrame()

for date in date_list:
    response = requests.get(f"https://api.clubelo.com/{date}")

    club_elo_df = pd.read_csv(io.StringIO(response.content.decode('utf-8')))
    club_elo_df['Rank'] = club_elo_df.index + 1
    club_elo_df['Updated'] = date
    
    dfs.append(club_elo_df)
    time.sleep(2)
    # merged_df.append(club_elo_df)
    
res = pd.concat(dfs)



KeyboardInterrupt: 

In [13]:

# Now we want to change club name according to clubs.csv and also change the index column name
# columns = [x.lower() for x in res.columns]
new_rows = ['rank', 'name', 'country', 'level', 'elo', 'from', 'to', 'updated']

res.columns = new_rows
# res = res[0:].reset_index(drop=True)  # Remove the first row and reset index
# 
res.head()


,rank,name,country,level,elo,from,to,updated
0,1,Liverpool,ENG,1,2088.261963,2020-01-30,2020-02-01,2020-01-31
1,2,Man City,ENG,1,2002.820190,2020-01-22,2020-02-02,2020-01-31
2,3,Barcelona,ESP,1,1964.985352,2020-01-26,2020-02-02,2020-01-31
3,4,Bayern,GER,1,1951.214600,2020-01-26,2020-02-01,2020-01-31
4,5,Juventus,ITA,1,1908.358643,2020-01-27,2020-02-02,2020-01-31


## Changing name of club elos to that of clubs df
So rn, names are mixed up (Man city instead of Manchester City) etc
We have to find a way to change this name into that of clubs df

In [55]:
test_df = res.head(600)
test_df.tail()

,rank,name,country,level,elo,from,to,updated
595,596,Zeta Golubovci,MNT,0,1076.796143,2019-08-14,2020-08-18,2020-01-31
596,597,Vitosha Bistritsa,BUL,1,1075.892090,2019-12-16,2020-02-14,2020-01-31
597,598,Valletta,MLT,0,1074.086792,2019-08-16,2020-08-19,2020-01-31
598,599,Mladost,MNT,0,1070.409912,2019-08-14,2020-07-01,2020-01-31
599,600,Rukh Brest,BLR,1,1070.054199,2020-01-01,2020-03-20,2020-01-31


In [57]:
# Change name to that in transfer mrkt dataset


# NOTE: This wont work for jupyter so we are using sth else for now
# # Get the absolute path of the current script (jupyter dir)
# BASE_DIR = Path(__file__).resolve().parent

# # Build the path to the data directory
# DATA_DIR = BASE_DIR / 'data'
BASE_DIR = Path.cwd()

# Build the path to the data directory
DATA_DIR = BASE_DIR.parents[0] / 'transfer_data' 
CLUBS_FILE = 'clubs.csv'
# res.to_csv(DATA_DIR / 'club_elos.csv')


filepath=os.path.join(DATA_DIR, CLUBS_FILE)
clubs_df=pd.read_csv(filepath,sep=",",encoding = "UTF-8")


filepath = os.path.join(DATA_DIR, 'player_valuations.csv')
player_valuations_df = pd.read_csv(filepath, sep=',', encoding="UTF-8")


def filter_by_season(dataset,season):
    # Filter for year 2023 and months from August to December
    year1 = (dataset['date'].dt.year == season) & (dataset['date'].dt.month >= 8)
    
    # Filter for year 2024 and months from January to June
    year2 = (dataset['date'].dt.year == (season + 1)) & (dataset['date'].dt.month <=6)
    
    filtered_data = dataset[year1 | year2]
    
    return filtered_data

player_valuations_df['date'] = pd.to_datetime(player_valuations_df['date'])
# NOTE: Fix this later
player_valuations = filter_by_season(player_valuations_df , 2020)

Lets get clubs total market value
by doing sth

In [58]:
player_valuations_df = player_valuations_df[player_valuations_df['current_club_id'].isin(clubs_df['club_id'])].groupby('current_club_id')['market_value_in_eur'].sum().reset_index()

In [59]:
clubs_valuations = pd.merge(clubs_df, player_valuations_df, how='left', left_on='club_id', 
                         right_on='current_club_id')
clubs_valuations = clubs_valuations.drop('total_market_value', axis=1)


,club_id,club_code,name,domestic_competition_id,squad_size,average_age,foreigners_number,foreigners_percentage,national_team_players,stadium_name,stadium_seats,net_transfer_record,coach_name,last_season,filename,url,current_club_id,market_value_in_eur
0,105,sv-darmstadt-98,SV Darmstadt 98,L1,27,25.6,13,48.1,1,Merck-Stadion am Böllenfalltor,17810,+€3.05m,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/sv-darmstadt-9...,105.0,9.770750e+08
1,11127,ural-ekaterinburg,Ural Yekaterinburg,RU1,30,26.5,11,36.7,3,Yekaterinburg Arena,23000,+€880k,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/ural-ekaterinb...,11127.0,1.670130e+09
2,114,besiktas-istanbul,Beşiktaş Jimnastik Kulübü,TR1,31,25.6,16,51.6,7,Tüpraş Stadyumu,42445,€-18.65m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/besiktas-istan...,114.0,7.548115e+09
3,12,as-rom,Associazione Sportiva Roma,IT1,26,25.7,19,73.1,18,Olimpico di Roma,73261,€-65.60m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/as-rom/startse...,12.0,1.296258e+10
4,148,tottenham-hotspur,Tottenham Hotspur Football Club,GB1,25,25.3,18,72.0,16,Tottenham Hotspur Stadium,62850,€-93.55m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/tottenham-hots...,148.0,1.383472e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434,724,fc-volendam,FC Volendam,NL1,25,23.4,6,24.0,2,Kras Stadion,7384,+€800k,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/fc-volendam/st...,724.0,1.368750e+08
435,800,atalanta-bergamo,Atalanta Bergamasca Calcio S.p.a.,IT1,26,26.8,15,57.7,19,Gewiss Stadium,21747,+€18.05m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/atalanta-berga...,800.0,9.063925e+09
436,85465,karpaty-lviv-2020-,FK Karpaty Lviv,UKR1,28,24.8,6,21.4,1,Ukraina,28051,€-1.48m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/karpaty-lviv-2...,85465.0,1.153100e+08
437,979,moreirense-fc,Moreirense Futebol Clube,PO1,24,25.8,16,66.7,3,Estádio C. J. de Almeida Freitas,6150,+€2.10m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/moreirense-fc/...,979.0,1.272450e+09


Now since we got total market value of clubs, let's combine market value - elo relationship and naming relation ship to find out the best match

In [ ]:
# Normalise ELO and market value
from sklearn.preprocessing import MinMaxScaler

# Normalize market values
def normalize_column(column):
    scaler = MinMaxScaler()
    return scaler.fit_transform(column.values.reshape(-1, 1))

player_valuations['normalised_market_value'] = normalize_column(player_valuations['market_value_in_eur'])
test_df['normalised_elo'] = normalize_column((test_df['elo']))

# Lets drop NaN values from test df
test_df = test_df[test_df['elo'].notna()]
clubs_valuations = clubs_valuations[clubs_valuations['market_value_in_eur'].notna()]

In [84]:
import re
# Function to clean the club names
def clean_club_name(name):
    # Define common terms to be removed
    common_terms = ['fc', 'football club', 'club de futbol', 'cf', 'sc', 'ac', 'sv']
    
    # Join terms into a regex pattern and remove them from the name
    pattern = '|'.join(common_terms)
    cleaned_name = re.sub(pattern, '', name, flags=re.IGNORECASE).strip()
    
    return cleaned_name

test_df['cleaned_name'] = test_df['name'].apply(clean_club_name)
clubs_valuations['cleaned_name'] = clubs_valuations['name'].apply(clean_club_name)


In [86]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sentence_transformers import SentenceTransformer

# Check for NaN values in the relevant columns and handle them
assert test_df['elo'].isnull().sum() == 0, "NaN values found in test_df['elo']"
assert clubs_valuations['market_value_in_eur'].isnull().sum() == 0, "NaN values found in clubs_valuations['market_value_in_eur']"


# Initialize the model for name similarity
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Compute cosine similarity for names
name_embeddings_elo = model.encode(test_df['cleaned_name'].tolist())
name_embeddings_valuations = model.encode(clubs_valuations['cleaned_name'].tolist())
cosine_similarities = cosine_similarity(name_embeddings_elo, name_embeddings_valuations)

# Normalize market values
scaler = MinMaxScaler()
clubs_valuations['normalized_market_value'] = scaler.fit_transform(clubs_valuations['market_value_in_eur'].values.reshape(-1, 1))
test_df['normalized_elo'] = scaler.fit_transform(test_df['elo'].values.reshape(-1, 1))

# Ensure both normalized values are not empty
assert not test_df['normalized_elo'].isnull().any(), "NaN values in normalized ELO"
assert not clubs_valuations['normalized_market_value'].isnull().any(), "NaN values in normalized market value"

# Compute market value similarities
market_values_elo = test_df['normalized_elo'].values
market_values_valuations = clubs_valuations['normalized_market_value'].values

# Market value similarity calculation using ratios
market_value_similarities = (market_values_elo[:, None] + market_values_valuations) / (1 + abs(market_values_elo[:, None] - market_values_valuations))

# Adjust the weights for name similarity and market value similarity
name_similarity_weight = 0.95  # Give more importance to name similarity
market_value_similarity_weight = 0.05  # Less importance to ELO/market value similarity

# Combine the scores with different weights
combined_scores = (name_similarity_weight * cosine_similarities) + (market_value_similarity_weight * market_value_similarities)

# Get the best matches
best_matches = combined_scores.argmax(axis=1)

# Create a DataFrame to show matches
results = pd.DataFrame({
    'Club (ELO)': test_df['name'],
    'Best Match (Valuations)': clubs_valuations['name'].iloc[best_matches].values,
    'Combined Score': combined_scores.max(axis=1)
})

# Display the results
results


/usr/local/Caskroom/miniconda/base/envs/footy/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


,Club (ELO),Best Match (Valuations),Combined Score
0,Liverpool,Liverpool FC,1.013018
1,Man City,Manchester City Football Club,0.648383
2,Barcelona,Futbol Club Barcelona,0.863718
3,Bayern,FC Bayern München,0.941661
4,Juventus,Juventus Football Club,1.014516
...,...,...,...
595,Zeta Golubovci,Venezia Football Club,0.482235
596,Vitosha Bistritsa,Vitória Setúbal FC,0.599435
597,Valletta,Associazione Calcio Monza,0.480057
598,Mladost,Odense Boldklub,0.446983


In [71]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from sklearn.preprocessing import MinMaxScaler

# Load your dataframes (assumed to be already loaded: test_df, clubs_valuations)
# Normalize ELO in test_df
scaler = MinMaxScaler()
test_df = test_df.copy()  # Create a copy to avoid SettingWithCopyWarning
test_df['normalized_elo'] = scaler.fit_transform(test_df[['elo']])

# Normalize market values in clubs_valuations
clubs_valuations = clubs_valuations.copy()  # Create a copy to avoid SettingWithCopyWarning
clubs_valuations['normalized_market_value'] = scaler.fit_transform(clubs_valuations[['market_value_in_eur']])

# Load a pre-trained model for sentence similarity
model = SentenceTransformer('paraphrase-MiniLM-L6-v2') 

# Encode club names
test_embeddings = model.encode(test_df['name'].tolist(), convert_to_tensor=True)
clubs_embeddings = model.encode(clubs_valuations['name'].tolist(), convert_to_tensor=True)

# Function to get the best match considering ELO and market value
def get_best_match(club_embedding, clubs_embeddings, clubs_valuations, test_row):
    cosine_scores = util.pytorch_cos_sim(club_embedding, clubs_embeddings)
    best_match_idx = cosine_scores.argmax().item()

    # Retrieve the best match's normalized market value and ELO
    matched_market_value = clubs_valuations.iloc[best_match_idx]['normalized_market_value']
    
    # Avoid division by zero
    if matched_market_value == 0:
        return clubs_valuations.iloc[best_match_idx]['name'], 0

    # Calculate a combined score (you can adjust weights as necessary)
    combined_score = cosine_scores[0][best_match_idx].item() * (test_row['normalized_elo'] / matched_market_value)

    return clubs_valuations.iloc[best_match_idx]['name'], combined_score

# Apply the function to get matched names and scores
results = test_df.apply(
    lambda row: get_best_match(test_embeddings[row.name], clubs_embeddings, clubs_valuations, row),
    axis=1
)

# Create new columns for matched club names and combined scores
test_df[['matched_club_name', 'combined_score']] = pd.DataFrame(results.tolist(), index=test_df.index)

# Merge the DataFrames based on the matched club names
merged_df = pd.merge(test_df, clubs_valuations, left_on='matched_club_name', right_on='name', how='left')


/usr/local/Caskroom/miniconda/base/envs/footy/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [72]:
merged_df

,rank,name_x,country,level,elo,from,to,updated,matched_club_name,matched_market_value,...,stadium_name,stadium_seats,net_transfer_record,coach_name,last_season,filename,url,current_club_id,market_value_in_eur,normalized_market_value
0,1,Liverpool,ENG,1,2088.261963,2020-01-30,2020-02-01,2020-01-31,Liverpool FC,1.767968e+10,...,Anfield,60725,+€5.00m,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/fc-liverpool/s...,31.0,1.767968e+10,0.672772
1,2,Man City,ENG,1,2002.820190,2020-01-22,2020-02-02,2020-01-31,Hull City,1.067300e+09,...,MKM Stadium,25586,+€7.50m,NaN,2016,../data/raw/transfermarkt-scraper/2016/clubs.j...,https://www.transfermarkt.co.uk/hull-city/star...,3008.0,1.067300e+09,0.039148
2,3,Barcelona,ESP,1,1964.985352,2020-01-26,2020-02-02,2020-01-31,Futbol Club Barcelona,2.514720e+10,...,Olímpic Lluís Companys,55926,€-19.40m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/fc-barcelona/s...,131.0,2.514720e+10,0.957596
3,4,Bayern,GER,1,1951.214600,2020-01-26,2020-02-01,2020-01-31,FC Bayern München,2.164805e+10,...,Allianz Arena,75024,€-67.65m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/fc-bayern-munc...,27.0,2.164805e+10,0.824132
4,5,Juventus,ITA,1,1908.358643,2020-01-27,2020-02-02,2020-01-31,Juventus Football Club,1.754785e+10,...,Allianz Stadium,41507,€-61.30m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/juventus-turin...,506.0,1.754785e+10,0.667744
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,596,Zeta Golubovci,MNT,0,1076.796143,2019-08-14,2020-08-18,2020-01-31,Spartak Vladikavkaz (-2020),1.388700e+08,...,Republican Stadium Spartak,32464,+-0,NaN,2012,../data/raw/transfermarkt-scraper/2012/clubs.j...,https://www.transfermarkt.co.uk/spartak-vladik...,2703.0,1.388700e+08,0.003736
596,597,Vitosha Bistritsa,BUL,1,1075.892090,2019-12-16,2020-02-14,2020-01-31,Vitória Setúbal FC,7.039250e+08,...,Estádio do Bonfim,18642,+-0,NaN,2019,../data/raw/transfermarkt-scraper/2019/clubs.j...,https://www.transfermarkt.co.uk/vitoria-setuba...,1085.0,7.039250e+08,0.025288
597,598,Valletta,MLT,0,1074.086792,2019-08-16,2020-08-19,2020-01-31,Associazione Calcio Monza,3.208075e+09,...,Brianteo,17102,+€18.50m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/ac-monza/start...,2919.0,3.208075e+09,0.120800
598,599,Mladost,MNT,0,1070.409912,2019-08-14,2020-07-01,2020-01-31,Odense Boldklub,7.966750e+08,...,Nature Energy Park,15790,+€916k,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/odense-boldklu...,173.0,7.966750e+08,0.028826


IDK BELOW


In [51]:
from fuzzywuzzy import process

def match_club_names(name, club_names):
    """
    Finds the best matching club name from a list of club names using fuzzy matching.
    """
    best_match, score = process.extractOne(name, club_names)
    return best_match  # Return the best matching club name


from fuzzywuzzy import fuzz

def fuzzy_match_score(name1, name2):
    """
    Computes a fuzzy match score between two club names.
    The score is a value between 0 and 1, with 1 being a perfect match.
    """
    score = fuzz.ratio(name1, name2) / 100  # Normalize the score to a range of 0 to 1
    return score

def get_market_value(club_name, clubs_df):
    """
    Retrieves the market value of the club from the DataFrame.
    If the club name does not exist, you can choose to return a default or estimated value.
    """
    try:
        # Assuming 'name' and 'market_value' columns exist in clubs_df
        market_value = clubs_df.loc[clubs_df['name'] == club_name, 'market_value_in_eur'].values[0]
    except IndexError:
        market_value = None  # Handle case where no match is found (optional: assign default value)
    
    return market_value


In [54]:
clubs_df

,club_id,club_code,name,domestic_competition_id,total_market_value,squad_size,average_age,foreigners_number,foreigners_percentage,national_team_players,stadium_name,stadium_seats,net_transfer_record,coach_name,last_season,filename,url,processed_name
0,105,sv-darmstadt-98,SV Darmstadt 98,L1,NaN,27,25.6,13,48.1,1,Merck-Stadion am Böllenfalltor,17810,+€3.05m,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/sv-darmstadt-9...,sv darmstadt 98
1,11127,ural-ekaterinburg,Ural Yekaterinburg,RU1,NaN,30,26.5,11,36.7,3,Yekaterinburg Arena,23000,+€880k,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/ural-ekaterinb...,ural yekaterinburg
2,114,besiktas-istanbul,Beşiktaş Jimnastik Kulübü,TR1,NaN,31,25.6,16,51.6,7,Tüpraş Stadyumu,42445,€-18.65m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/besiktas-istan...,beşiktaş jimnastik kulübü
3,12,as-rom,Associazione Sportiva Roma,IT1,NaN,26,25.7,19,73.1,18,Olimpico di Roma,73261,€-65.60m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/as-rom/startse...,associazione sportiva roma
4,148,tottenham-hotspur,Tottenham Hotspur Football Club,GB1,NaN,25,25.3,18,72.0,16,Tottenham Hotspur Stadium,62850,€-93.55m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/tottenham-hots...,tottenham hotspur
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434,724,fc-volendam,FC Volendam,NL1,NaN,25,23.4,6,24.0,2,Kras Stadion,7384,+€800k,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/fc-volendam/st...,volendam
435,800,atalanta-bergamo,Atalanta Bergamasca Calcio S.p.a.,IT1,NaN,26,26.8,15,57.7,19,Gewiss Stadium,21747,+€18.05m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/atalanta-berga...,atalanta bergamasca calcio s.p.a.
436,85465,karpaty-lviv-2020-,FK Karpaty Lviv,UKR1,NaN,28,24.8,6,21.4,1,Ukraina,28051,€-1.48m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/karpaty-lviv-2...,fk karpaty lviv
437,979,moreirense-fc,Moreirense Futebol Clube,PO1,NaN,24,25.8,16,66.7,3,Estádio C. J. de Almeida Freitas,6150,+€2.10m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/moreirense-fc/...,moreirense futebol clube


In [52]:
# Apply combined matching logic
def get_combined_score(club_elo_name, clubs_df):
    best_match_score = 0
    best_match_club = None
    
    # Iterate over clubs_df names
    for _, club_df_row in clubs_df.iterrows():
        club_name = club_df_row['name']
        
        # Calculate the individual similarity scores
        fuzz_score = fuzz.ratio(club_elo_name, club_name)
        token_sort_score = fuzz.token_sort_ratio(club_elo_name, club_name)
        
        # Assuming 'club_value' is in both dataframes, and normalize it
        club_value_df = club_df_row['']
        # club_value_elo = clubs_elo[clubs_elo['name'] == club_elo_name]['club_value'].values[0]
        # value_diff = 1 - abs(club_value_df - club_value_elo) / max(club_value_df, club_value_elo)
        
        # Combine the scores
        combined_score = fuzz_score * 0.4 + token_sort_score * 0.4 + 
        
        # Track the best match
        if combined_score > best_match_score:
            best_match_score = combined_score
            best_match_club = club_name
    
    return best_match_club, best_match_score

# Apply this to the clubs_elo dataframe
clubs_elo['best_match'], clubs_elo['best_match_score'] = zip(*clubs_elo['name'].apply(lambda x: get_combined_score(x, clubs_df)))

# Sort by the best match score
clubs_elo = clubs_elo.sort_values('best_match_score', ascending=False)



ValueError: too many values to unpack (expected 2)

In [26]:
# Step 1: Apply the matching function, using .loc[] to avoid SettingWithCopyWarning
test_df.loc[:, 'matched_club_name'] = test_df['name'].apply(lambda x: match_club_names(x, clubs_df['name']))

# Step 2: Merge the DataFrames based on the matched club names
merged_df = pd.merge(test_df, clubs_df, left_on='matched_club_name', right_on='name', how='left')


In [27]:
merged_df

,rank,name_x,country,level,elo,from,to,updated,matched_club_name,club_id,...,foreigners_number,foreigners_percentage,national_team_players,stadium_name,stadium_seats,net_transfer_record,coach_name,last_season,filename,url
0,1,Liverpool,ENG,1,2088.261963,2020-01-30,2020-02-01,2020-01-31,Liverpool FC,31.0,...,21.0,77.8,22.0,Anfield,60725.0,+€5.00m,NaN,2023.0,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/fc-liverpool/s...
1,2,Man City,ENG,1,2002.820190,2020-01-22,2020-02-02,2020-01-31,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Barcelona,ESP,1,1964.985352,2020-01-26,2020-02-02,2020-01-31,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Bayern,GER,1,1951.214600,2020-01-26,2020-02-01,2020-01-31,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Juventus,ITA,1,1908.358643,2020-01-27,2020-02-02,2020-01-31,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,596,Zeta Golubovci,MNT,0,1076.796143,2019-08-14,2020-08-18,2020-01-31,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
596,597,Vitosha Bistritsa,BUL,1,1075.892090,2019-12-16,2020-02-14,2020-01-31,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
597,598,Valletta,MLT,0,1074.086792,2019-08-16,2020-08-19,2020-01-31,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
598,599,Mladost,MNT,0,1070.409912,2019-08-14,2020-07-01,2020-01-31,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


---
Below is IDK whats gonig on


In [8]:
# Now we want to change club name according to clubs.csv and also change the index column name
# columns = [x.lower() for x in res.columns]
new_rows = ['rank', 'name', 'country', 'level', 'elo', 'from', 'to', 'updated']

res.columns = new_rows
# res = res[0:].reset_index(drop=True)  # Remove the first row and reset index
# 
res.head()


,rank,name,country,level,elo,from,to,updated
0,1,Liverpool,ENG,1,2088.261963,2020-01-30,2020-02-01,2020-01-31
1,2,Man City,ENG,1,2002.820190,2020-01-22,2020-02-02,2020-01-31
2,3,Barcelona,ESP,1,1964.985352,2020-01-26,2020-02-02,2020-01-31
3,4,Bayern,GER,1,1951.214600,2020-01-26,2020-02-01,2020-01-31
4,5,Juventus,ITA,1,1908.358643,2020-01-27,2020-02-02,2020-01-31


In [9]:
# Change name to that in transfer mrkt dataset


# NOTE: This wont work for jupyter so we are using sth else for now
# # Get the absolute path of the current script (jupyter dir)
# BASE_DIR = Path(__file__).resolve().parent

# # Build the path to the data directory
# DATA_DIR = BASE_DIR / 'data'
BASE_DIR = Path.cwd()

# Build the path to the data directory
DATA_DIR = BASE_DIR.parents[0] / 'transfer_data' 
CLUBS_FILE = 'clubs.csv'
# res.to_csv(DATA_DIR / 'club_elos.csv')


filepath=os.path.join(DATA_DIR, CLUBS_FILE)
clubs_df=pd.read_csv(filepath,sep=",",encoding = "UTF-8")

In [10]:
clubs_df.head(5)

,club_id,club_code,name,domestic_competition_id,total_market_value,squad_size,average_age,foreigners_number,foreigners_percentage,national_team_players,stadium_name,stadium_seats,net_transfer_record,coach_name,last_season,filename,url
0,105,sv-darmstadt-98,SV Darmstadt 98,L1,NaN,27,25.6,13,48.1,1,Merck-Stadion am Böllenfalltor,17810,+€3.05m,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/sv-darmstadt-9...
1,11127,ural-ekaterinburg,Ural Yekaterinburg,RU1,NaN,30,26.5,11,36.7,3,Yekaterinburg Arena,23000,+€880k,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/ural-ekaterinb...
2,114,besiktas-istanbul,Beşiktaş Jimnastik Kulübü,TR1,NaN,31,25.6,16,51.6,7,Tüpraş Stadyumu,42445,€-18.65m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/besiktas-istan...
3,12,as-rom,Associazione Sportiva Roma,IT1,NaN,26,25.7,19,73.1,18,Olimpico di Roma,73261,€-65.60m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/as-rom/startse...
4,148,tottenham-hotspur,Tottenham Hotspur Football Club,GB1,NaN,25,25.3,18,72.0,16,Tottenham Hotspur Stadium,62850,€-93.55m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/tottenham-hots...


In [12]:
# Matching name
clubs_df[clubs_df['name'].str.contains('Liverpool')]

,club_id,club_code,name,domestic_competition_id,total_market_value,squad_size,average_age,foreigners_number,foreigners_percentage,national_team_players,stadium_name,stadium_seats,net_transfer_record,coach_name,last_season,filename,url
44,31,fc-liverpool,Liverpool FC,GB1,NaN,27,25.2,21,77.8,22,Anfield,60725,+€5.00m,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/fc-liverpool/s...


In [43]:
#

,rank,name,country,level,elo,from,to,updated,club_id
0,1,Liverpool,ENG,1,2088.261963,2020-01-30,2020-02-01,2020-01-31,-1
1,2,Man City,ENG,1,2002.820190,2020-01-22,2020-02-02,2020-01-31,-1
2,3,Barcelona,ESP,1,1964.985352,2020-01-26,2020-02-02,2020-01-31,-1
3,4,Bayern,GER,1,1951.214600,2020-01-26,2020-02-01,2020-01-31,-1
4,5,Juventus,ITA,1,1908.358643,2020-01-27,2020-02-02,2020-01-31,-1


['rank', 'club', 'country', 'level', 'elo', 'from', 'to', 'updated']

In [ ]:

# NOTE: This wont work for jupyter so we are using sth else for now
# # Get the absolute path of the current script (jupyter dir)
# BASE_DIR = Path(__file__).resolve().parent

# # Build the path to the data directory
# DATA_DIR = BASE_DIR / 'data'
BASE_DIR = Path.cwd()

# Build the path to the data directory
DATA_DIR = BASE_DIR.parents[0] / 'transfer_data' 

res.to_csv(DATA_DIR / 'club_elos.csv')